In [89]:
import pandas as pd
import numpy as np
import word2vec
import datetime
import pickle
from collections import defaultdict, OrderedDict
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
from ast import literal_eval


from typing import List

import random
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from recommender.aRecommender import ARecommender #class

from recommendation.recommendation import Recommendation #class
from recommendation.resultOfRecommendation import ResultOfRecommendation #class

from datasets.ratings import Ratings #class

class RecommenderW2V(ARecommender):

        #def __init__(self, arguments: List[Argument]):
        #if type(arguments) is not Arguments:
        #    raise ValueError("Argument arguments is not type Arguments.")

        #self._arguments:List[Argument] = arguments
        #Tyv*le stepane, to musi mit kazda key-value pair svoji vlastni tridu??? Kdo ma ty definice hledat? Na tohle ti staci obycejny dictionary
        #jak rad rikam, na tohle ti sere bilej tesak - kdyztak si to prepis, ja budu pouzivat dictionary

    # ratingsSum:Dataframe<(userId:int, movieId:int, ratings:int, timestamp:int)>
    def __init__(self, arguments):
        #arguments je dictionary, povinny parametr je cesta k souboru s CB daty
        self._arguments = arguments
        #"../../../../data/cbDataOHE.txt" nebo "../../../../data/cbDataTFIDF.txt"
        #self.w2vDataPath = self._arguments["w2vDataPath"]
        self.trainVariant = self._arguments["trainVariant"]
        self.userProfiles = {}
        
    def getTrainVariant(self, trainDF):
        if self.trainVariant == "all":
            return trainDF
        elif self.trainVariant == "positive":
            return trainDF.loc[trainDF[Ratings.COL_RATING] >=4]
        elif self.trainVariant == "posneg":
            trainDF[Ratings.COL_MOVIEID].loc[trainDF[Ratings.COL_RATING] < 4] = -trainDF[Ratings.COL_MOVIEID]
            return trainDF
           
        
        
    def train(self, historyDF:pd.DataFrame, ratingsDF:pd.DataFrame, usersDF:pd.DataFrame, itemsDF:pd.DataFrame):                
        if type(ratingsDF) is not DataFrame:
            raise ValueError("Argument trainRatingsDF is not type DataFrame.")
            
            
        t = self.getTrainVariant(ratingsDF)
        t[Ratings.COL_MOVIEID] = t[Ratings.COL_MOVIEID].astype("str")
        t_sequences = t.groupby(Ratings.COL_USERID)[Ratings.COL_MOVIEID].apply(" ".join)
        print(t_sequences)
        #t_sequences.set_index(Ratings.COL_USERID, inplace=True)
        w2vTrainData = t_sequences.values.tolist()
        
        w = 3
        e = 64
        model, rev_dict, dictionary = word2vec.word2vecRun(w,e, w2vTrainData)
        dictionary = dict([((int(i),j) if i !="RARE" else (-1,j)) for i,j in dictionary.items() ])
        rev_dict = dict(zip(dictionary.values(), dictionary.keys()))
        self.model = model
        self.dictionary = dictionary
        self.rev_dict = rev_dict
        
        # ratingsSum:Dataframe<(userId:int, movieId:int, ratings:int, timestamp:int)>

        self.ratingsGroupDF = t.groupby(Ratings.COL_USERID)[Ratings.COL_MOVIEID]
        userProfileDF = self.ratingsGroupDF.aggregate(lambda x: list(x))
        self.userProfiles = userProfileDF.to_dict()


    def update(self, ratingsUpdateDF:pd.DataFrame):
        #ratingsUpdateDF has only one row
        ratingsUpdateDF = self.getTrainVariant(ratingsUpdateDF)
        if ratingsUpdateDF.shape[0] > 0:
            row = ratingsUpdateDF.iloc[0]
            rating = row[Ratings.COL_RATING]
            userID = row[Ratings.COL_USERID]
            objectID = row[Ratings.COL_MOVIEID]            
            userTrainData = self.userProfiles.get(userID,[])
            userTrainData.append(objectID)
            self.userProfiles[userID] = userTrainData
        
    
    def resolveUserProfile(self, userProfileStrategy, userTrainData):
        objectIDs = [int(i) for i in userTrainData]
        w2vObjects = [self.dictionary[i] for i in objectIDs if i in self.dictionary]   
        
        rec = userProfileStrategy
        print(rec)
        if (len(w2vObjects) > 0):
            if (rec == "mean") | (rec == "max"):
                weights = [1.0] * len(w2vObjects)
            elif rec == "last":
                w2vObjects = w2vObjects[-1:]
                weights = [1.0]
            elif rec == "window3":
                w2vObjects = w2vObjects[-3:]
                weights = [1 / len(w2vObjects) * i for i in range(1, (len(w2vObjects) + 1))]
            elif rec == "window5":
                w2vObjects = w2vObjects[-5:]
                weights = [1 / len(w2vObjects) * i for i in range(1, (len(w2vObjects) + 1))]
            elif rec == "window10":
                w2vObjects = w2vObjects[-10:]
                weights = [1 / len(w2vObjects) * i for i in range(1, (len(w2vObjects) + 1))]
            
            if rec == "max":
                agg = np.max
            else:
                agg = np.mean
                
            print((w2vObjects, weights, agg))
            return (w2vObjects, weights, agg)
        

        return ([],[],"")

        
    def recommend(self, userID:int, numberOfItems:int, userProfileStrategy):
        userTrainData = self.userProfiles.get(userID,[])
        w2vObjects, weights, aggregation = self.resolveUserProfile(userProfileStrategy,userTrainData)
        simList = []
        
        #provedu agregaci dle zvolené metody
        if len(w2vObjects)>0:        
            embeds = self.model[w2vObjects]
            results = 1 - pairwise_distances(embeds, self.model, metric="cosine")
            
            weights = np.asarray(weights).reshape((-1, 1))
            results = results * weights
            results = aggregation(results, axis=0)
            
            print(type(results))
            #check for a variant with negative preference (only positive objects recommended)
            #approximative solution - might result in less objects
            resultList = (-results).argsort()[0:(numberOfItems*3)] 
            resultingOIDs = [self.rev_dict[i] for i in resultList if self.rev_dict[i] > 0] 
            resultingOIDs = resultingOIDs[0:numberOfItems]
            resultList = [self.dictionary[i] for i in resultingOIDs]

            #print(results[resultList])
            
            #normalize scores into the unit vector (for aggregation purposes)
            #!!! tohle je zasadni a je potreba provest normalizaci u vsech recommenderu - teda i pro most popular!
            finalScores = results[resultList]
            finalScores = normalize(np.expand_dims(finalScores, axis=0))[0,:]
            
            return ResultOfRecommendation(resultingOIDs, finalScores.tolist())

        return ResultOfRecommendation([], [])

In [90]:
rec = RecommenderW2V({"trainVariant":"posneg"})#zahrnuje negativni rating
#rec = RecommenderW2V({"trainVariant":"positive"})#pouze pozitivni rating

In [91]:
from typing import List

from pandas.core.series import Series #class

from recommender.description.recommenderDescription import RecommenderDescription #class

from recommender.aRecommender import ARecommender #class

from recommender.recommenderTheMostPopular import RecommenderTheMostPopular #class
from recommender.dummy.recommenderDummyRedirector import RecommenderDummyRedirector #class

from datasets.ratings import Ratings #class
from datasets.rating import Rating #class

from datasets.users import Users #class

from portfolio.portfolioDescription import PortfolioDescription #class
from portfolio.portfolio import Portfolio #class

from aggregation.aggregationDescription import AggregationDescription #class
from aggregation.aggrDHont import AggrDHont #class

from recommendation.resultOfRecommendation import ResultOfRecommendation #class

from simulation.evaluationTool.simplePositiveFeedback import SimplePositiveFeedback #class

import numpy as np
import pandas as pd

from pandas.core.frame import DataFrame #class

import os

In [92]:
ratingsFile: str = "../../.." + os.sep + "data" + os.sep +  "ratings.dat"

ratingsDF: DataFrame = pd.read_csv(ratingsFile, sep=';', header=None)
ratingsDF.columns = [Ratings.COL_USERID, Ratings.COL_MOVIEID, Ratings.COL_RATING, Ratings.COL_TIMESTAMP]
ratingsDFTrain = ratingsDF.iloc[0:50000]

In [93]:
rec.train(pd.DataFrame(),ratingsDFTrain,pd.DataFrame(),pd.DataFrame())

C:\Users\lpesk\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lpesk\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


userId
1      1193 -661 -914 3408 2355 -1197 1287 2804 594 9...
2      1357 3068 1537 -647 2194 648 2268 -2628 -1103 ...
3      3421 -1641 -648 1394 -3534 104 2735 1210 -1431...
4      3468 -1210 2951 1214 1036 260 2028 480 -1196 1...
5      2987 2333 1175 -39 -288 2337 1535 1392 -2268 -...
                             ...                        
327    3789 1248 -1179 1250 -3865 1251 1252 1254 722 ...
328    -648 589 -736 1198 -667 -3020 -1356 -3107 -137...
329    2987 -1175 1179 -719 -648 -3861 1320 1321 -386...
330    -3863 1250 -3793 1256 3798 -3948 1262 -2138 11...
331    1179 647 648 3863 -2125 -2126 1180 3868 3869 -...
Name: movieId, Length: 331, dtype: object
Creating Dictionary
Actual vocabulary size:5351
Loss at step 10000 : 0.732938826084137
Loss at step 20000 : 0.5157715082168579
Loss at step 30000 : 0.6451883912086487
Loss at step 40000 : 1.3929405212402344
Loss at step 50000 : 1.725585699081421
Nearest to 2858: -2008, 3521, -1531, 1686, -1194,
Nearest to 480: -3623, -2852

In [94]:
len(rec.userProfiles[331])

144

In [95]:
ratingsDFUpdate = ratingsDF.iloc[50001:50002]
ratingsDFUpdate.iloc[0,1]=5000 #test jak si poradi s neexistujicimi vstupy
ratingsDFUpdate

C:\Users\lpesk\.conda\envs\tf\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\lpesk\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,userId,movieId,rating,timestamp
50001,331,5000,2,984152511


In [96]:
rec.update(ratingsDFUpdate)

C:\Users\lpesk\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [97]:
len(rec.userProfiles[331])

145

In [98]:
rec.model.shape

(5351, 64)

In [99]:
rec.recommend(331, 50, "max")

max
([238, 1215, 54, 1084, 1628, 539, 2855, 341, 1185, 504, 1546, 34, 242, 43, 521, 1464, 372, 474, 312, 794, 252, 413, 1109, 4152, 3983, 1623, 623, 137, 1094, 819, 28, 1650, 204, 26, 109, 2, 84, 5, 360, 11, 1393, 920, 703, 605, 22, 2539, 216, 3012, 1532, 2805, 5344, 2096, 2152, 2712, 110, 3878, 538, 72, 187, 674, 826, 2275, 388, 194, 812, 231, 1776, 1346, 2117, 3322, 2351, 36, 394, 715, 3368, 295, 714, 89, 2797, 345, 3828, 3379, 5345, 860, 531, 1940, 232, 145, 1598, 50, 164, 5346, 547, 1519, 41, 3135, 2688, 254, 155, 563, 4154, 3516, 325, 142, 1854, 1909, 456, 1748, 151, 844, 1612, 5347, 5348, 1007, 2636, 485, 797, 264, 2776, 1237, 934, 5349, 1116, 1238, 224, 117, 3274, 593, 764, 503, 780, 1298, 42, 2033, 1131, 179, 1276, 5350, 1369, 2415, 95, 2961, 1112, 495], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1